In [7]:
import numpy as np
import os
import torch
from torch_geometric.nn import GCNConv, SAGEConv
import sys
sys.path.append("../")
from models.concurrency.baselines.gcn_graph_embedding import load_all_data


In [3]:
data_dir = "/Users/ziniuw/Desktop/research/perf_prediction/workload-performance/pmodel_data/imdb/"
dataset = "sample-plan"
n_run_id = 4
print("Loading all data graphs")
all_features, all_edge_idx, all_edge_weight, all_labels, all_root_idx_bit_map = load_all_data(data_dir, dataset, n_run_id)
n_graphs = len(all_features)
print(f"In total, {n_graphs} number of graphs")

Loading all data graphs
In total, 835 number of graphs


In [8]:
gcn = SAGEConv(all_features[0].shape[-1], 64)

In [10]:
pred = gcn(all_features[1], all_edge_idx[1])
np.sum(np.isnan(pred.detach().numpy()))

0

In [28]:
path = "/Users/ziniuw/Desktop/research/perf_prediction/workload-performance/pmodel_data/imdb/"
dataset = "sample-plan-0-0"
vmatrix = np.load(os.path.join(path, dataset + "-nodes.npy"))
ematrix = np.load(os.path.join(path, dataset + "-edges.npy"))

In [31]:
def load_data_from_matrix(vmatrix, ematrix, val_len=0):
    idx_features_labels = vmatrix
    n_example = len(vmatrix)
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = idx_features_labels[:, -1].astype(float)
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    edges_unordered = ematrix[:, :-1]
    edges = np.array(
        list(map(idx_map.get, edges_unordered.flatten())), dtype=np.int32
    ).reshape(edges_unordered.shape)

    # modified begin.
    edges_value = ematrix[:, -1:]
    adj = sp.coo_matrix(
        (edges_value[:, 0], (edges[:, 0], edges[:, 1])),
        shape=(n_example, n_example),
        dtype=np.float32,
    )

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0]))

    operator_num = adj.shape[0]
    train_len = 1 - val_len
    idx_train = range(int(train_len * operator_num))
    # print("idx_train", idx_train)
    idx_val = range(int(train_len * operator_num), operator_num)
    features = torch.FloatTensor(np.array(features.todense()))
    adj = sparse_mx_to_torch_sparse_tensor(adj)
    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    labels = labels.astype(np.float32)
    labels = torch.from_numpy(labels)
    labels.unsqueeze(1)

    return adj, features, labels, idx_train, idx_val

In [32]:
adj, features, labels, idx_train, idx_val = load_data_from_matrix(
        vmatrix, ematrix, 0
    )

NameError: name 'sp' is not defined

In [33]:
vmatrix[:, 0]

array([0.000e+00, 1.000e+00, 2.000e+00, ..., 1.075e+03, 1.076e+03,
       1.077e+03])

In [34]:
len(vmatrix)

1078

In [49]:
feature = torch.from_numpy(vmatrix[:, 1:-1]).type(torch.FloatTensor)
edge_idx = torch.from_numpy(ematrix[:, :-1].T).type(torch.int64)
edge_weight = torch.from_numpy(ematrix[:, -1]).type(torch.FloatTensor)

In [50]:
gcn(feature, edge_idx, edge_weight)

tensor([[ 2.1032e-01, -4.5925e-01, -2.7318e-01,  ..., -1.0206e-01,
          8.8229e-02, -1.6933e-01],
        [ 4.2127e-01, -4.9595e-01, -4.3956e-01,  ..., -2.4059e-01,
          3.6633e-01, -7.5833e-02],
        [ 4.0685e-01, -5.4034e-01, -5.0996e-01,  ..., -2.9821e-01,
          1.2428e-01, -2.7391e-01],
        ...,
        [ 9.5358e+01, -9.4131e+01, -7.6838e+01,  ..., -7.1039e+01,
          2.4031e+01, -7.3422e+01],
        [ 1.1096e+02, -1.0175e+02, -8.2907e+01,  ..., -7.8919e+01,
          3.2658e+01, -8.3240e+01],
        [ 9.1925e+01, -8.4296e+01, -6.8753e+01,  ..., -6.5439e+01,
          2.7140e+01, -6.8776e+01]], grad_fn=<AddBackward0>)

In [51]:
edge_weight

tensor([ 1.0000,  1.0000,  1.0000,  ..., -0.0713, -0.0412, -0.0412])